In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

In [ ]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"     #Quantized model
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
model.eval()

comment = "Great content, thank you!"
prompt=f'''[INST] {comment} [/INST]'''    #As per docs

inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

In [ ]:
intstructions_string = f"""Engage-GPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–Engage-GPT'. \
Engage-GPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""

prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

prompt = prompt_template(comment)
print(prompt)

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt")

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

outputs = model.generate(
    input_ids=inputs["input_ids"].to("cuda"),
    attention_mask=inputs["attention_mask"].to("cuda"),
    pad_token_id=tokenizer.pad_token_id,
    max_new_tokens=140
)

# Decode
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])


In [ ]:
model.train()

model.gradient_checkpointing_enable()

model = prepare_model_for_kbit_training(model)

In [ ]:
# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

model.print_trainable_parameters()

In [ ]:
data = load_dataset("shawhin/shawgpt-youtube-comments")

In [ ]:
def tokenize_function(examples):
    text = examples["example"]

    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

tokenized_data = data.map(tokenize_function, batched=True)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)


In [ ]:
lr = 2e-4
batch_size = 4
num_epochs = 10
training_args = transformers.TrainingArguments(
    output_dir= "PRAKHAR-gpt-ft",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",

)

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    args=training_args,
    data_collator=data_collator
)


model.config.use_cache = False
trainer.train()

model.config.use_cache = True